# Reading with GDAL for Python

This notebook demonstrates some minimal raster/spatial operations with AVIRIS-NG files using the Python interface to GDAL.

<h2 id="tocheading">Table of Contents</h2>       
<br>
<div id="toc"></div>     

*This next cell calls a script to generate a TOC. It will display above when this notebook is opened in the Jupyter environment. Ignore.*

In [1]:
%%javascript
$.getScript('scripts/tocgen.js')

<IPython.core.display.Javascript object>

## Workflow

### Imports and example file
Import requirements. Minimal packages:

In [2]:
import numpy as np
from osgeo import gdal, osr

#### Unzip and open tarfile
AVIRIS-NG data files are distributed in a zipped tarfile. See the document `<link>` for more details. 

You can unzip the example file with the `tarfile` module like this:
```python
import tarfile

with tarfile.open("data/ang20180814t224053rfl.tar.gz", "r:gz") as tar:
    tar.extractall()
```

See what's inside:

In [3]:
import glob
glob.glob("data/ang20180814t224053_rfl_v2r2/*")

['data/ang20180814t224053_rfl_v2r2\\ang20180814t224053_corr_v2r2_img',
 'data/ang20180814t224053_rfl_v2r2\\ang20180814t224053_corr_v2r2_img.hdr',
 'data/ang20180814t224053_rfl_v2r2\\ang20180814t224053_h2o_v2r2_img',
 'data/ang20180814t224053_rfl_v2r2\\ang20180814t224053_h2o_v2r2_img.hdr',
 'data/ang20180814t224053_rfl_v2r2\\ang20180814t224053_README_v2r2.txt']

### Reading an image
Open the example reflectance file:

In [4]:
img = 'data/ang20180814t224053_rfl_v2r2/ang20180814t224053_corr_v2r2_img'
hdr = 'data/ang20180814t224053_rfl_v2r2/ang20180814t224053_corr_v2r2_img.hdr'

ds = gdal.Open(img)

Open the header file:

In [5]:
with open('data/ang20180814t224053_rfl_v2r2/ang20180814t224053_corr_v2r2_img.hdr','r') as f:
    hdr = [ln.strip() for ln in f.readlines()]

### Raster image shape
GDAL makes accessing the shape of the image pretty easy:

In [6]:
bands = ds.RasterCount # band count
cols = ds.RasterXSize  # col count
rows = ds.RasterYSize  # row count

print("bands:\t"+str(bands)) 
print("cols:\t"+str(cols))
print("rows:\t"+str(rows))

bands:	425
cols:	637
rows:	4207


### Geotransform
The geotransform is a tuple of parameters (6 floats) that defines the transformation from each pixel's x,y position in the image to its projected position in the reference coordinate system (affine transformation). 

**This website gives a clear, thorough explanation of affine transforms and their use in GIS:**         
http://www.quantdec.com/GIS/affine.htm

**More info on geographic transformation and GDAL's raster data model:**       
https://www.gdal.org/gdal_datamodel.html

Get the tuple with `ds.GetGeoTransform()`:

In [7]:
ds.GetGeoTransform()

(447779.369091,
 4.177675425873858,
 2.9252398253903347,
 7185907.49943,
 2.9252398253903347,
 -4.177675425873858)

#### Calculate pixel coordinates
The coordinates for each pixel are calculated like:
```
GT = ( 447779.369091,                        <-   0 x minimum (top left)
       4.177675425873858,                    <-   1 x resolution
       2.9252398253903347,                   <-   2 x rotation
       7185907.49943,                        <-   3 y maximum (top left)
       2.9252398253903347,                   <-   4 y rotation
      -4.177675425873858 )                   <-   5 y resolution

Xpixel                                       <-  x/column index of pixel
Yline                                        <-  y/row index of pixel

Xproj  = GT(0) + Xpixel*GT(1) + Yline*GT(2)  <-  x coordinate
Yproj  = GT(3) + Xpixel*GT(4) + Yline*GT(5)  <-  y coordinate
```

Calculate the projected coordinates for the pixel at the bottom right corner `(column == 637, row == 4207)`:

In [8]:
# transformation parameters
GT = ds.GetGeoTransform()

# pixel indices are base 0
Xpixel, Yline = (cols - 1, rows - 1)

# x,y calculate w affine transform equation
Xproj = GT[0] + Xpixel*GT[1] + Yline*GT[2]
Yproj = GT[3] + Xpixel*GT[4] + Yline*GT[5]

print("x (m):\t"+str(Xproj))
print("y (m):\t"+str(Yproj))

x (m):	462739.92936744756
y (m):	7170196.649117722


### Coordinate arrays

To make a CF compliant netCDF we need:
* 1-dimensional arrays of x and y coordinates (2)
* 2-dimensional arrays of lon and lat coordinates (2)

First generate the arrays of x and y coordinates. Unpack the geotransform into its component parts and make 1-d arrays with their origins at the top left corner of the raster like:
```
Coordinates calculated over interval equal to their resolution:
    
    xpos(i):  x_origin_in_meters + i*x_resolution
    ypos(i):  y_origin_in_meters + i*y_resolution

x_coordinate_array = xpos(sequence 0 to number_of_columns-1)
y_coordinate_array = ypos(sequence 0 to number_of_rows-1)

```

In [9]:
# get the raster geotransform as its component parts
xmin, xres, xrot, ymax, yrot, yres = ds.GetGeoTransform()

# generate coordinate arrays
xarr = np.array([xmin+i*xres for i in range(0,cols)])
yarr = np.array([ymax+i*yres for i in range(0,rows)])

print("x[0]: \t"+str(xarr[0]))
print("y[0]:\t"+str(yarr[0]))

x[0]: 	447779.369091
y[0]:	7185907.49943


### Get 2d arrays of latitudes and longitudes

Use the `osr` package to get the proj4 string from the input raster dataset:

In [10]:
native_srs = osr.SpatialReference()
native_srs.ImportFromWkt(ds.GetProjection())
proj4 = native_srs.ExportToProj4()

proj4

'+proj=utm +zone=3 +datum=WGS84 +units=m +no_defs '

`pyproj` is the Python interface to libproj. Use pyproj to transform the first pixel's `utm x,y -->> lon,lat`:

In [11]:
from pyproj import Proj, transform

inproj = Proj(proj4)
outproj = Proj(init="epsg:4326")
lon, lat = transform(inproj, outproj, xarr[0], yarr[0])

lon, lat

(-166.0989579203195, 64.79362088575823)

Permute the x and y arrays with `np.meshgrid`:

In [12]:
xarr2d, yarr2d = np.meshgrid(xarr, yarr)

print("Each array now has this shape:\t"+str(xarr2d.shape))

Each array now has this shape:	(4207, 637)


Flatten both arrays and pass to the `pyproj.transform` function:

In [13]:
lonarr, latarr = transform(
    inproj,               # input raster srs
    outproj,              # output raster srs
    xarr2d.flatten(),     # flat 2d array of x coordinates
    yarr2d.flatten())     # flat 2d array of y coordinates

print("lon[0]:\t"+str(lonarr[0]))
print("lat[0]:\t"+str(latarr[0]))

lon[0]:	-166.0989579203195
lat[0]:	64.79362088575823


Return the flat arrays to the shape of the raster:

In [14]:
lonarr2d = lonarr.reshape(xarr2d.shape)
latarr2d = latarr.reshape(yarr2d.shape)

lonarr2d.shape

(4207, 637)